# In this notebook, we're going to cover some of the most fundamental concepts of tensor using TensorFlow

More specifically,

* Indtroduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors and NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow(or TPUs)
* Exercise to try for yourself

## Indtroduction to Tensors

In [14]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.5.0


In [15]:
# Creating tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [16]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [17]:
# Create a vector 
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [18]:
# Check the dimensions of vector
vector.ndim

1

In [19]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[2, 3],
                     [3, 2]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [3, 2]], dtype=int32)>

In [20]:
matrix.ndim

2

In [21]:
matrix_2 = tf.constant([[3., 2.],
                        [4., 5.],
                        [1., 2.]], dtype=tf.float16)
matrix_2

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[3., 2.],
       [4., 5.],
       [1., 2.]], dtype=float16)>

In [22]:
matrix_2.ndim

2

In [23]:
# let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [2, 3, 4]],
                      [[3, 4, 5],
                       [4, 5, 6]],
                      [[5, 6, 7],
                       [6,7,8]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[1, 2, 3],
        [2, 3, 4]],

       [[3, 4, 5],
        [4, 5, 6]],

       [[5, 6, 7],
        [6, 7, 8]]], dtype=int32)>

In [24]:
tensor.ndim

3

*Note:* 
* Scalar: a single number
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-D array of numbers
* Tensor: an n-D array of numbers (when `n` can be any number, 0-D tensor is scalar, a 1-D tensor is a vector)

### Creating tensors with `tf.Variable`

In [25]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([1, 2])
unchangeable_tensor = tf.constant([1, 2])

changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([1, 2], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>)

In [26]:
# Let's try to change oone of the elements in our changeable tensor
changeable_tensor[0] = 2

TypeError: ignored

In [ ]:
# Let's try it with .assign()
changeable_tensor[0].assign(2)
changeable_tensor

In [ ]:
# Let's try to change unchangeable tensor
unchangeable_tensor[0] = 2

In [ ]:
unchangeable_tensor[0].assign(2)
unchangeable_tensor

### Creating random tensors

Random tensors are tensors of some abitrary size which contain random numbers.


In [ ]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42)
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

random_1, random_2, random_1 == random_2

### Shuffle the order of elements in a tensor

**Note:** Valuable for when you want to shuffle your data so the inherent order doesn't affect learning.

https://www.tensorflow.org/guide/random_numbers

In [ ]:
# create a tensor
not_shuffled = tf.constant([[1,2],
                           [2, 3],
                           [3, 4]])

# shuffle a tensor
tf.random.shuffle(not_shuffled)

In [ ]:
tf.random.set_seed(42) # global level
tf.random.shuffle(not_shuffled, seed=42) # operation level

https://www.tensorflow.org/api_docs/python/tf/random/set_seed

In order to get same shuffled results we need to set both `Global seed` and `Operation seed` with same value

**Rule 4:** If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence.

In [ ]:
tf.random.set_seed(42) # global level
tf.random.shuffle(not_shuffled, seed=42) # operation level

### Other ways to make tensors

In [ ]:
# Create a tensor with ones
tf.ones([3,4])

In [ ]:
# Create a tensor with zeros
tf.zeros(shape=(3,4))

### Turn NumPy array into a tensor

The main diffrence between NumPy array and TensorFlow tensor is that the tensoe can be run on GPU(for much faster numerical computing)

In [27]:
# Create a NumPy array
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

**Note:** 
* X = tf.constant(some_matrix) --> Capital for matrix or tensor
* y = tf.constant(vector) --> non-capital for vector

In [28]:
# Convert NumPy array into tensor
A = tf.constant(numpy_A)

# Reshape the tensor
B = tf.constant(numpy_A, shape=(2, 3, 4)) # 2*3*4 = 24
A, B

(<tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>,
 <tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>)

### Getting information from tensors

Tensor attributes:
* Shape
* Rank
* Axis or dimension
* Size

In [29]:
# Create a Rank 4 tensor (4 Dimensions)
rank_4_tensor = tf.zeros(shape=(2,3,4,5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [30]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [31]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor) # size =2*3*4*5 = 120

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

### Indexing tensors

Tensors can be indexed just like Python lists.

In [32]:
# Get the first 2 elements of each dimensions
rank_4_tensor[:2,:2,:2,:2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [33]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1,:1,:1]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [34]:
# Create a rank 2 tensor (2 Dimensions)
rank_2_tensor = tf.constant([[1,2],
                             [3,4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [35]:
# Get the last two items of the each row
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 4], dtype=int32)>

In [36]:
# Add an extra dimension to rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[1],
        [2]],

       [[3],
        [4]]], dtype=int32)>

In [37]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # expand final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[1],
        [2]],

       [[3],
        [4]]], dtype=int32)>

In [38]:
# Expand zero axis
tf.expand_dims(rank_2_tensor, axis=0)

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[1, 2],
        [3, 4]]], dtype=int32)>

### Manipulating Tensors (tensor operations)

**Basic Operations**

`+`, `-`, `*`,`/`

In [39]:
# You add values to a tensor using addition operator
tensor = tf.constant([[10, 8],
                     [12, 5]])
tensor + 10 # Original tensor is unchanged

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 18],
       [22, 15]], dtype=int32)>

In [40]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  80],
       [120,  50]], dtype=int32)>

In [41]:
# Substraction 
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -2],
       [ 2, -5]], dtype=int32)>

In [42]:
tensor / 2

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[5. , 4. ],
       [6. , 2.5]])>

In [43]:
# We can use TensorFlow built-in function 
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  80],
       [120,  50]], dtype=int32)>

In [44]:
tf.add(tensor, 2)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[12, 10],
       [14,  7]], dtype=int32)>

In [45]:
tf.subtract(tensor, 2)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 8,  6],
       [10,  3]], dtype=int32)>

In [46]:
tf.divide(tensor, 2)

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[5. , 4. ],
       [6. , 2.5]])>

### Matrix Multiplication

In Machine learning, matrix multiplication is one of the most common tensor operations.

In [47]:
# Matrix multiplication in tensorflow
print(tensor)
tf.matmul(tensor, tensor)

tf.Tensor(
[[10  8]
 [12  5]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[196, 120],
       [180, 121]], dtype=int32)>

In [48]:
tensor * tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  64],
       [144,  25]], dtype=int32)>

In [49]:
# Matrix multiplication with Python operator `@`
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[196, 120],
       [180, 121]], dtype=int32)>

In [50]:
# Create a tensor (3,2) tensor
X = tf.constant([[1,2],
                 [2,3],
                 [3,4]])

# create a one more tensor
Y = tf.constant([[4,5],
                 [5,6],
                 [6,7]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [2, 3],
        [3, 4]], dtype=int32)>, <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[4, 5],
        [5, 6],
        [6, 7]], dtype=int32)>)

In [51]:
# try matrix multiply on tensors of same shape
X@Y

InvalidArgumentError: ignored

In [52]:
tf.matmul(X, Y)

InvalidArgumentError: ignored

In [53]:
tf.matmul(X, tf.transpose(Y)) # tf.transpose is to transpose the matrix

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[14, 17, 20],
       [23, 28, 33],
       [32, 39, 46]], dtype=int32)>

In [54]:
# Another way to do tensor multiplication
X @ tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[16, 17, 19],
       [26, 28, 31],
       [36, 39, 43]], dtype=int32)>

* transpose - flips rows
* shape - rearrages elements

### The DOT product
Matrix multiplication is alos referred to as dot product

You can perform matrix multiplication using:

* tf.matmul()
* tf.tensordot()

In [55]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [2, 3],
        [3, 4]], dtype=int32)>, <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[4, 5],
        [5, 6],
        [6, 7]], dtype=int32)>)

In [56]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[32, 38],
       [47, 56]], dtype=int32)>

In [57]:
# Perform matrix maultiplication  between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[14, 17, 20],
       [23, 28, 33],
       [32, 39, 46]], dtype=int32)>

In [58]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y,shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[16, 17, 19],
       [26, 28, 31],
       [36, 39, 43]], dtype=int32)>

In [59]:
# Check the values of Y,  reshape Y and transposed Y
print("Normal Y:")
print(Y, "\n") 
print(tf.reshape(Y, shape=(2,3)), "\n")
print("Y trnsposed:")
print(tf.transpose(Y))

Normal Y:
tf.Tensor(
[[4 5]
 [5 6]
 [6 7]], shape=(3, 2), dtype=int32) 

tf.Tensor(
[[4 5 5]
 [6 6 7]], shape=(2, 3), dtype=int32) 

Y trnsposed:
tf.Tensor(
[[4 5 6]
 [5 6 7]], shape=(2, 3), dtype=int32)


### Change the data type of a Tensor

In [60]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.2, 2.1])
B.dtype

tf.float32

In [61]:
C = tf.constant([1,2])
C.dtype

tf.int32

In [62]:
# Change the float32 to float16 ( reduced precision)
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.2, 2.1], dtype=float16)>,
 tf.float16)

In [63]:
# Change form int32 to float32
E = tf.cast(C, dtype=tf.float32)
E

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

In [64]:
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1., 2.], dtype=float16)>

### Aggregating Tensors

In [65]:
# Getting the absolute values
D = tf.constant([-1, -2])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([-1, -2], dtype=int32)>

In [66]:
# Get the absolute values
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>

Let's go through the following forms of aggregations:
* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of tensor

In [67]:
# Create a random tensor with values between 0 to 100 of size 10
F = tf.constant(np.random.randint(0, 100, size=50))
F

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([29, 32, 12, 19, 36, 31, 77, 94,  3, 52, 47, 56,  0, 52, 72, 75, 49,
       29,  2, 43, 93, 94, 60, 80, 12, 37, 18, 65, 54, 68, 12, 30, 28, 13,
       87, 76, 95, 51, 55, 47, 81, 18, 49, 96, 75, 53, 51,  4, 20, 12])>

In [68]:
tf.size(F), F.shape, F.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [69]:
tf.reduce_min(F), tf.reduce_max(F), tf.reduce_mean(F), tf.reduce_sum(F)

(<tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=96>,
 <tf.Tensor: shape=(), dtype=int64, numpy=46>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2344>)

In [70]:
import tensorflow_probability as tfp
tfp.stats.variance(F)

<tf.Tensor: shape=(), dtype=int64, numpy=791>

In [71]:
tf.math.reduce_variance(tf.cast(F, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=791.14557>

In [72]:
tf.math.reduce_std(tf.cast(F, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=28.12731>

### Find the positional maximun and minimum

In [75]:
# Create a new tensor for finding positional minimum and amximum
G = tf.random.set_seed(42)
G = tf.random.uniform(shape=[50])
G

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [76]:
# Fina the positional maximum
tf.argmax(G)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [84]:
# Index on our largest value position
G[tf.argmax(G)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [79]:
# Find the max value of F
tf.reduce_max(G)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [85]:
# Check for equality
G[tf.argmax(G)] == tf.reduce_max(G)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [86]:
# Find the positional minimum
tf.argmin(G)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [87]:
G[tf.argmin(G)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

### Squeezing a tensor (removing all single dimensions)

In [89]:
# Create a tensor 
tf.random.set_seed(42)
H = tf.constant(tf.random.uniform(shape=[50]), shape=(1,1,1,1,50))
H

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [90]:
H.shape

TensorShape([1, 1, 1, 1, 50])

In [91]:
H_squeezed = tf.squeeze(H)
H_squeezed, H_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>, TensorShape([50]))

### One-hot encoding

In [92]:
# create a list of indices
some_list = [0,1,2,3] 

# One hot encode our list of indices
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [94]:
# Specify custom values for one hot encoding (just for fun)
tf.one_hot(some_list, depth=4, on_value="Hello...", off_value="World!!!")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'Hello...', b'World!!!', b'World!!!', b'World!!!'],
       [b'World!!!', b'Hello...', b'World!!!', b'World!!!'],
       [b'World!!!', b'World!!!', b'Hello...', b'World!!!'],
       [b'World!!!', b'World!!!', b'World!!!', b'Hello...']], dtype=object)>

### Few math functions

Squaring, log, square root

In [96]:
I = tf.range(1, 10)
I

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [97]:
# square it
tf.square(I)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [100]:
# Find the square root
tf.math.sqrt(tf.cast(I, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [103]:
#  Find the log
tf.math.log(tf.cast(I, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

In [105]:
tf.math.exp(tf.cast(I, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([2.7182817e+00, 7.3890562e+00, 2.0085537e+01, 5.4598148e+01,
       1.4841316e+02, 4.0342877e+02, 1.0966332e+03, 2.9809580e+03,
       8.1030840e+03], dtype=float32)>

In [106]:
tf.math.count_nonzero(I)

<tf.Tensor: shape=(), dtype=int64, numpy=9>

In [108]:
tf.math.erf(tf.cast(I, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.8427007, 0.9953223, 0.999978 , 1.       , 1.       , 1.       ,
       1.       , 1.       , 1.       ], dtype=float32)>

### Tensors and NumPy

In [109]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([1., 3., 5.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([1., 3., 5.])>

In [112]:
# Convert our tensor back to a NumPy array
np.array(J), type(np.array(J))

(array([1., 3., 5.]), numpy.ndarray)

In [113]:
# Convert tensor J into NumPy array
J.numpy(), type(J.numpy())

(array([1., 3., 5.]), numpy.ndarray)

In [114]:
# The default type of each are  slightly different 
numpy_J = tf.constant(np.array([1., 3., 5.]))
tensor_J = tf.constant([1., 3., 5.])

# Check the datatype
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)